# Gallery Example: M/Par/1 Queue

This example demonstrates an M/Pareto/1 queueing system:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Pareto distribution (heavy-tailed)
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

Pareto service times create heavy-tailed behavior with occasional very long service times.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mpar1():
    """Create M/Par/1 queueing model"""
    model = Network('M/Par/1')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    source.set_arrival(oclass1, Exp(1))
    # Pareto service with mean=0.5 and SCV=64 (very heavy tail)
    queue.set_service(oclass1, Pareto.fit_mean_and_scv(0.5, 64))
    
    # Block 3: topology
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_route(oclass1, queue, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_mpar1()

## About Pareto Service Times

The Pareto distribution:
- Is a power-law distribution with heavy tail
- Commonly used for modeling file sizes, network traffic, etc.
- Can have extremely high variability (SCV >> 1)

With SCV = 64:
- Variance is 64 times the squared mean
- Occasional very long service times occur
- Creates significant queue buildup during long services

This model demonstrates the impact of heavy-tailed service on queueing performance.

In [ ]:
# Solve with Fluid solver (CTMC may struggle with heavy tails)
print("\n=== Solver Results ===")

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)